In [2]:
import libcst as cst
import pytest
from pathlib import Path

from stubber.codemod.enrich import package_from_path


class FunctionDocstringVisitor(cst.CSTVisitor):
    def __init__(self):
        self.function_docstrings = {}

    def visit_FunctionDef(self, node: cst.FunctionDef):
        docstring = None
        if node.body.body and isinstance(node.body.body[0], cst.SimpleStatementLine):
            first_statement = node.body.body[0]
            if isinstance(first_statement.body[0], cst.Expr) and isinstance(
                first_statement.body[0].value, cst.SimpleString
            ):
                docstring = first_statement.body[0].value.value.strip("\"'")
        self.function_docstrings[node.name.value] = docstring


def get_function_docstrings(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        module = cst.parse_module(file.read())

    visitor = FunctionDocstringVisitor()
    module.visit(visitor)
    return visitor.function_docstrings

In [3]:
# Define the path to the type stub file



def test_function_docstrings(file: Path):
    docstrings = get_function_docstrings(file)
    id = package_from_path(file) or file.stem


    for func_name, docstring in docstrings.items():
        yield id, func_name, docstring

        # Assert that the function has a docstring (or not, depending on your requirement)

        # assert docstring is not None, f"Function '{func_name}' does not have a docstring"

In [4]:
stub_file = Path(
    "d:/mypython/micropython-stubber/repos/micropython-stubs/stubs/micropython-v1_24_1-frozen/rp2/GENERIC/onewire.pyi"
)

assert stub_file.exists(), f"Stub file '{stub_file}' does not exist"
list(test_function_docstrings(stub_file))

[('onewire', '__init__', None),
 ('onewire', 'reset', None),
 ('onewire', 'readbit', None),
 ('onewire', 'readbyte', None),
 ('onewire', 'readinto', None),
 ('onewire', 'writebit', None),
 ('onewire', 'writebyte', None),
 ('onewire', 'write', None),
 ('onewire', 'select_rom', None),
 ('onewire', 'scan', None),
 ('onewire', '_search_rom', None),
 ('onewire', 'crc8', None)]

In [5]:
base = Path("d:/mypython/micropython-stubber/repos/micropython-stubs/micropython-reference")
all_stubfiles = base.glob("**/*.pyi")

all_info = []
for stub_file in all_stubfiles:
    all_info.extend(list(test_function_docstrings(stub_file)))

In [7]:
import sqlite3

# Connect to an in-memory SQLite database
conn = sqlite3.connect("docstrings.db")
cursor = conn.cursor()

# Create a table to store the information
cursor.execute(
    """
CREATE TABLE docstrings (
    module TEXT,
    function_name TEXT,
    docstring TEXT
)
"""
)




# Insert the data from all_info into the table
cursor.executemany(
    """
INSERT INTO docstrings (module, function_name, docstring)
VALUES (?, ?, ?)
""",
    all_info,
)

# Commit the transaction
conn.commit()

# Verify the data is inserted
cursor.execute("SELECT * FROM docstrings")
rows = cursor.fetchall()
for row in rows:
    print(row)

('array.__init__', '__init__', '\n        Create array with elements of given type. Initial contents of the\n        array are given by *iterable*. If it is not provided, an empty\n        array is created.\n        ')
('array.__init__', 'append', '\n        Append new element *val* to the end of array, growing it.\n        ')
('array.__init__', 'extend', '\n        Append new elements as contained in *iterable* to the end of\n        array, growing it.\n        ')
('array.__init__', '__getitem__', '\n        Indexed read of the array, called as ``a[index]`` (where ``a`` is an ``array``).\n        Returns a value if *index* is an ``int`` and an ``array`` if *index* is a slice.\n        Negative indices count from the end and ``IndexError`` is thrown if the index is\n        out of range.\n\n        **Note:** ``__getitem__`` cannot be called directly (``a.__getitem__(index)`` fails) and\n        is not present in ``__dict__``, however ``a[index]`` does work.\n        ')
('array.__init__

In [23]:
# Close the connection
conn.close()

In [ ]:
def test_function_docstrings(file):
    docstrings = get_function_docstrings(file)

    for func_name, docstring in docstrings.items():
        print(f"Function '{func_name}' docstring: {docstring}")
        # Assert that the function has a docstring (or not, depending on your requirement)
        assert docstring is not None, f"Function '{func_name}' does not have a docstring"


if __name__ == "__main__":
    pytest.main([__file__])